In [ ]:
import os
import re
import pandas as pd

# ================== 1️⃣ 路径配置 (请根据您的实际路径修改) ==================
# 假设所有CSV文件都在这个BASE_DIR下
BASE_DIR = "/mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/csv"

# 输入文件
dicom_path = os.path.join(BASE_DIR, "dicom_info.csv")
calc_train_path = os.path.join(BASE_DIR, "calc_case_description_train_set.csv")
mass_train_path = os.path.join(BASE_DIR, "mass_case_description_train_set.csv")
calc_test_path = os.path.join(BASE_DIR, "calc_case_description_test_set.csv")
mass_test_path = os.path.join(BASE_DIR, "mass_case_description_test_set.csv")

# 输出文件
OUTPUT_TRAIN_PATH = "./train_cleaned.csv"
OUTPUT_TEST_PATH = "./test_cleaned.csv"

# ================== 2️⃣ 辅助函数 (来自代码一) ==================

def extract_uids(path):
    """从路径中提取 StudyInstanceUID / SeriesInstanceUID"""
    uids = re.findall(r"1\.3\.6\.1\.4\.1\.9590\.[\d\.]+", str(path))
    if len(uids) >= 2:
        return pd.Series({"StudyInstanceUID": uids[0], "SeriesInstanceUID": uids[1]})
    else:
        return pd.Series({"StudyInstanceUID": None, "SeriesInstanceUID": None})

def normalize_pathology(x):
    """标准化病理标签"""
    x = str(x).strip().upper()
    if "MALIGNANT" in x:
        return "MALIGNANT"
    elif "BENIGN_WITHOUT_CALLBACK" in x:
        return "BENIGN_WITHOUT_CALLBACK"
    elif "BENIGN" in x:
        return "BENIGN"
    return x

def merge_with_dicom(df, dicom, lesion_type):
    """根据 UID 匹配 dicom_info，返回包含图像路径的完整 DataFrame (来自代码一)"""
    df["lesion_type"] = lesion_type

    # 提取 UID
    uid_info = df["image file path"].apply(extract_uids)
    df = pd.concat([df, uid_info], axis=1)

    # 标准化标签
    df["pathology"] = df["pathology"].apply(normalize_pathology)

    # 过滤 full mammogram (来自代码一)
    dicom_full = dicom[dicom["SeriesDescription"].str.contains("full", case=False, na=False)]
    dicom_unique = dicom_full.drop_duplicates(subset=["StudyInstanceUID", "SeriesInstanceUID"], keep="first")

    merged = df.merge(
        dicom_unique, # 合并所有 dicom_info 列，以便后续清洗
        on=["StudyInstanceUID", "SeriesInstanceUID"],
        how="left"
    )

    before = len(merged)
    # 确定 'image_path' 列的正确名称 (合并后可能带后缀 _y)
    image_path_col = 'image_path_y' if 'image_path_y' in merged.columns else 'image_path'
    
    # ⭐【已修改】重命名为标准的最终路径列
    merged = merged.rename(columns={image_path_col: 'image_path_final'})
    
    # ⭐【已修改】使用新列名 dropna
    merged = merged.dropna(subset=['image_path_final']).reset_index(drop=True)
    
    after = len(merged)
    print(f"✅ {lesion_type}: 成功匹配 {after}/{before} 条记录 ({after/before:.1%})")

    return merged

# ================== 3️⃣ 预清洗函数 (来自代码二) ==================

def clean_calc_df(df):
    """应用代码二的策略清洗 calc_case_description"""
    print(f"清洗 calc_case_df (原始条数: {len(df)})")
    # 重命名 (代码二, 步骤 5)
    df = df.rename(columns={
        'calc type':'calc_type',
        'calc distribution':'calc_distribution',
        'image view':'image_view',
        'left or right breast':'left_or_right_breast',
        'breast density':'breast_density',
        'abnormality type':'abnormality_type'
    })
    
    # 填充缺失值 (代码二, 步骤 5)
    if 'calc_type' in df.columns:
        df['calc_type'] = df['calc_type'].bfill()
    if 'calc_distribution' in df.columns:
        df['calc_distribution'] = df['calc_distribution'].bfill()
        
    return df

def clean_mass_df(df):
    """应用代码二的策略清洗 mass_case_description"""
    print(f"清洗 mass_case_df (原始条数: {len(df)})")
    # 重命名 (代码二, 步骤 6)
    df = df.rename(columns={
        'mass shape':'mass_shape',
        'mass margins':'mass_margins',
        'image view':'image_view',
        'left or right breast':'left_or_right_breast',
        'abnormality type':'abnormality_type'
    })
    
    # 填充缺失值 (代码二, 步骤 6)
    if 'mass_shape' in df.columns:
        df['mass_shape'] = df['mass_shape'].bfill()
    if 'mass_margins' in df.columns:
        df['mass_margins'] = df['mass_margins'].bfill()
        
    return df

# ================== 4️⃣ 后清洗函数 (来自代码二) ==================

def post_merge_clean(df):
    """应用代码二的策略清洗合并后的 DataFrame"""
    
    # 填充缺失值 (代码二, 步骤 4)
    if 'SeriesDescription' in df.columns:
        df['SeriesDescription'] = df['SeriesDescription'].bfill()
    if 'Laterality' in df.columns:
        df['Laterality'] = df['Laterality'].bfill()

    # ⭐【已修改】将所有您不希望的列加入删除列表
    COLS_TO_DROP = [
        # Columns from code 2 (UIDs, dates, etc.)
        'PatientBirthDate','AccessionNumber','Columns','ContentDate','ContentTime',
        'PatientSex','ReferringPhysicianName','Rows','SOPClassUID',
        'SOPInstanceUID','StudyDate','StudyID','StudyInstanceUID','StudyTime',
        'InstanceNumber','SeriesInstanceUID','SeriesNumber',
        
        # Original paths and redundant columns
        'image file path', 
        'cropped image file path', 'ROI mask file path',
        'image_path_x', 'image_path', 
        
        # Expert features (from previous request)
        'calc_type', 'calc_distribution', 'mass_shape', 'mass_margins',
        
        # ⭐ 新增: (from this request)
        'patient_id',       # from calc/mass files
        'lesion_type',      # created by this script
        'PatientID',        # from dicom_info
        'PatientName',       # from dicom_info
        'abnormality_type',
        'pathology',
        'file_path',
        'Laterality',
        'PatientOrientation',
    ]
    
    # 只删除数据框中实际存在的列
    cols_exist = [col for col in COLS_TO_DROP if col in df.columns]
    df = df.drop(columns=cols_exist)
    
    return df

# ================== 5️⃣ 主执行流程 ==================

print("--- 1. 加载数据 ---")
dicom = pd.read_csv(dicom_path)
calc_train = pd.read_csv(calc_train_path)
mass_train = pd.read_csv(mass_train_path)
calc_test = pd.read_csv(calc_test_path)
mass_test = pd.read_csv(mass_test_path)

print(f"Dicom info 条数: {len(dicom)}")
print(f"Calc train 条数: {len(calc_train)}, Calc test 条数: {len(calc_test)}")
print(f"Mass train 条数: {len(mass_train)}, Mass test 条数: {len(mass_test)}")

print("\n--- 2. 预清洗 (代码二逻辑) ---")
calc_train_cleaned = clean_calc_df(calc_train)
mass_train_cleaned = clean_mass_df(mass_train)
calc_test_cleaned = clean_calc_df(calc_test)
mass_test_cleaned = clean_mass_df(mass_test)

print("\n--- 3. 合并Dicom (代码一逻辑) ---")
merged_calc_train = merge_with_dicom(calc_train_cleaned, dicom, "calcification")
merged_mass_train = merge_with_dicom(mass_train_cleaned, dicom, "mass")
merged_calc_test = merge_with_dicom(calc_test_cleaned, dicom, "calcification")
merged_mass_test = merge_with_dicom(mass_test_cleaned, dicom, "mass")

print("\n--- 4. 创建4分类标签 (代码一逻辑) ---")
label_map = {
    ("calcification", "MALIGNANT"): 0,
    ("calcification", "BENIGN"): 1,
    ("calcification", "BENIGN_WITHOUT_CALLBACK"): 1,
    ("mass", "MALIGNANT"): 2,
    ("mass", "BENIGN"): 3,
    ("mass", "BENIGN_WITHOUT_CALLBACK"): 3,
}

for df in [merged_calc_train, merged_mass_train, merged_calc_test, merged_mass_test]:
    df["label_4c"] = df.apply(
        lambda r: label_map.get((r["lesion_type"], r["pathology"]), None), axis=1
    )

print("--- 5. 合并 Train / Test 集 ---")
train_df = pd.concat([merged_calc_train, merged_mass_train], ignore_index=True)
test_df = pd.concat([merged_calc_test, merged_mass_test], ignore_index=True)

# 打乱训练集 (来自代码一)
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
# 测试集通常不需要打乱
test_df = test_df.reset_index(drop=True)


print(f"Train set 总样本数: {len(train_df)}")
print(f"Test set 总样本数: {len(test_df)}")

print("\n--- 6. 后清洗 (代码二逻辑) ---")
train_df_final = post_merge_clean(train_df)
test_df_final = post_merge_clean(test_df)

print(f"Train set 清洗后剩余列数: {len(train_df_final.columns)}")
print(f"Test set 清洗后剩余列数: {len(test_df_final.columns)}")

# 替换路径
# 1. 确保 image_path_final 列是字符串类型，以避免 .str 访问器出错
train_df_final['image_path_final'] = train_df_final['image_path_final'].astype(str)
test_df_final['image_path_final'] = test_df_final['image_path_final'].astype(str)

# 2. 执行字符串替换
train_df_final['image_path_final'] = train_df_final['image_path_final'].str.replace(
    "CBIS-DDSM", 
    "cbis-ddsm-breast-cancer-image-dataset"
)
test_df_final['image_path_final'] = test_df_final['image_path_final'].str.replace(
    "CBIS-DDSM", 
    "cbis-ddsm-breast-cancer-image-dataset"
)

print("\n--- 7. 保存结果 ---")
train_df_final.to_csv(OUTPUT_TRAIN_PATH, index=False)
test_df_final.to_csv(OUTPUT_TEST_PATH, index=False)

print(f"✅ 已保存训练集: {OUTPUT_TRAIN_PATH}")
print(f"✅ 已保存测试集: {OUTPUT_TEST_PATH}")

print("\n--- 最终训练集 标签分布 (label_4c) ---")
print(train_df_final["label_4c"].value_counts(dropna=False))

print("\n--- 最终训练集 示例数据 (已删除所有指定列) ---")
print(train_df_final.head())

In [2]:
import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from PIL import Image
from tqdm import tqdm

# ==============================
# 路径配置
# ==============================
# 1. 包含 train_cleaned.csv 和 test_cleaned.csv 的目录
DATASET_ROOT = "/mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/" 
BASE_IMAGE_PATH = "/mnt/hdd/jiazy/"
# 2. 处理后文件的输出目录
OUT_DIR = os.path.join(DATASET_ROOT, "features")
os.makedirs(OUT_DIR, exist_ok=True)

# 3. (已更新) 定义您的输入文件
TRAIN_CLEANED_CSV = os.path.join(DATASET_ROOT, "train_cleaned.csv")
TEST_CLEANED_CSV = os.path.join(DATASET_ROOT, "test_cleaned.csv")

print(f"数据源 (CSV): {DATASET_ROOT}")
print(f"输出目录 (Processed): {OUT_DIR}\n")

# ==============================
# 1️⃣ 加载已清理的 CSV
# ==============================
print("Step 1/11: 加载已清理的 train_cleaned.csv 和 test_cleaned.csv...")
if not os.path.exists(TRAIN_CLEANED_CSV) or not os.path.exists(TEST_CLEANED_CSV):
    print(f"🔴 错误：找不到 {TRAIN_CLEANED_CSV} 或 {TEST_CLEANED_CSV}")
    print("请确保 DATASET_ROOT 变量设置正确。")
    exit(1)

try:
    train_df = pd.read_csv(TRAIN_CLEANED_CSV)
    test_df  = pd.read_csv(TEST_CLEANED_CSV)
except Exception as e:
    print(f"🔴 错误：读取 CSV 失败: {e}")
    exit(1)

print(f"    Train DF (loaded): {len(train_df)} 行, Test DF (loaded): {len(test_df)} 行")

# ==============================
# 2️⃣ (已修改) 定义特征类型 (基于您的要求)
# ==============================
print("Step 2/11: 定义特征类型 (基于您的要求)...")
new_label_col = "label_4c"
image_path_col_final = "image_path_final" 

# 1. 来自您要求的非特征列
# (我们合并了您提供的列和脚本中原有的其他非特征列)
non_feature_cols_from_script = [
    "pathology", "abnormality type", "lesion_type",
    "patient_id", "file_path", "abnormality id"
]
non_feature_cols = list(set(
    [new_label_col, image_path_col_final] + non_feature_cols_from_script
))

# 2. 来自您要求的连续特征
final_continuous_cols = [
    'BitsAllocated', 'BitsStored', 'HighBit', 'LargestImagePixelValue', 
    'PixelRepresentation', 'SamplesPerPixel', 'SmallestImagePixelValue'
]

# 3. 来自您要求的类别特征
final_categorical_cols = [
    'left_or_right_breast', 'image_view', 'BodyPartExamined', 'ConversionType', 
    'Modality', 'PhotometricInterpretation', 'SecondaryCaptureDeviceManufacturer', 
    'SecondaryCaptureDeviceManufacturerModelName', 'SeriesDescription', 
    'SpecificCharacterSet', 'breast_density', 'assessment', 'subtlety'
]

# 确保这些列存在于 DataFrame 中，过滤掉不存在的列
all_feature_cols_requested = final_continuous_cols + final_categorical_cols
available_cols = set(train_df.columns)

final_continuous_cols = [col for col in final_continuous_cols if col in available_cols]
final_categorical_cols = [col for col in final_categorical_cols if col in available_cols]

missing_cols = [col for col in all_feature_cols_requested if col not in available_cols]
if missing_cols:
    print(f"    ⚠️ 警告: 您指定的一些特征列在 train_df.csv 中不存在，将被忽略: {missing_cols}")

print(f"    (新) 使用您指定的 {len(final_continuous_cols)} 个连续特征: {final_continuous_cols}")
print(f"    (新) 使用您指定的 {len(final_categorical_cols)} 个分类特征: {final_categorical_cols}")

# ==============================
# 3️⃣ (新) 删除在 train/test 中均只有一个唯一值的列
# ==============================
print("Step 3/11: 检查并删除 '恒定' 特征...")
cols_to_drop = []
all_feature_cols = final_continuous_cols + final_categorical_cols

for col in all_feature_cols:
    # 检查列是否存在，以防万一
    if col not in train_df.columns or col not in test_df.columns:
        continue
        
    try:
        # nunique() 默认不计算 NaN
        train_unique_count = train_df[col].nunique()
        test_unique_count = test_df[col].nunique()
        
        # 如果在 train 和 test 中，唯一值的数量都为 1 (或 0)
        if train_unique_count <= 1 and test_unique_count <= 1:
            cols_to_drop.append(col)
            print(f"    -> 标记为删除 (Train 唯一值: {train_unique_count}, Test 唯一值: {test_unique_count}): {col}")
            
    except Exception as e:
        print(f"    ⚠️ 检查 {col} 唯一值时出错: {e}")

# 从特征列表中移除这些列
final_continuous_cols = [col for col in final_continuous_cols if col not in cols_to_drop]
final_categorical_cols = [col for col in final_categorical_cols if col not in cols_to_drop]

if cols_to_drop:
    print(f"    ✅ 已从特征列表中移除 {len(cols_to_drop)} 个恒定列。")
else:
    print("    ✅ 未发现恒定特征列。")

def make_abs_path(path):
    # 如果路径是 NaN 或 None，保持原样
    if pd.isna(path):
        return path
    # 使用 os.path.join 智能地拼接基础路径和相对路径
    return os.path.join(BASE_IMAGE_PATH, str(path))

train_df[image_path_col_final] = train_df[image_path_col_final].apply(make_abs_path)
test_df[image_path_col_final] = test_df[image_path_col_final].apply(make_abs_path)

print(f"    路径转换完成。示例: {train_df[image_path_col_final].iloc[0]}")

# ==============================
# 4️⃣ (原 Step 3) 过滤标签/路径
# ==============================
print("Step 4/11: 过滤无效的标签/路径...")
# (您的 info 显示 2864 non-null，但这一步是确保安全的)
train_df.dropna(subset=[image_path_col_final, new_label_col], inplace=True)
test_df.dropna(subset=[image_path_col_final, new_label_col], inplace=True)
print(f"    过滤后剩余: Train {len(train_df)}, Test {len(test_df)}")

# ==============================
# 5️⃣ (原 Step 4) 填充特征中的缺失值 (NaN)
# ==============================
print("Step 5/11: 填充特征中的缺失值 (NaN)...")
# (使用 .loc 避免 FutureWarning)
print("-> 正在为 _TIP (无填补) 版本创建数据备份...")
train_df_tip = train_df.copy()
test_df_tip = test_df.copy()


for col in final_continuous_cols:
    mean_val = train_df[col].mean()
    train_df.loc[:, col] = train_df[col].fillna(mean_val)
    test_df.loc[:, col]  = test_df[col].fillna(mean_val)

for col in final_categorical_cols:
    train_df.loc[:, col] = train_df[col].fillna("MISSING")
    test_df.loc[:, col]  = test_df[col].fillna("MISSING")

# ==============================
# 6️⃣ (原 Step 5) 编码分类特征
# ==============================
print("Step 6/11: 编码分类特征...")
field_lengths = [] 
# 检查：如果删除了所有分类列，则跳过
if final_categorical_cols:
    full_df_imputed = pd.concat([train_df, test_df], ignore_index=True)

    for col in final_categorical_cols:
        full_df_imputed[col] = full_df_imputed[col].astype('category')
        codes = full_df_imputed[col].cat.codes
        cardinality = len(full_df_imputed[col].cat.categories)
        field_lengths.append(cardinality)
        train_df[col] = codes.iloc[:len(train_df)].values
        test_df[col]  = codes.iloc[len(train_df):].values
else:
    print("    (跳过：未找到分类特征)")

print("Step 6.B/11: 编码分类特征 [_TIP 版]...")
if final_categorical_cols:
    full_df_tip = pd.concat([train_df_tip, test_df_tip], ignore_index=True)
    for col in final_categorical_cols:
        full_df_tip[col] = full_df_tip[col].astype('category')
        # .cat.codes 会自动将 NaN 映射为 -1
        codes_tip = full_df_tip[col].cat.codes
        # (我们不需要为 TIP 版本保存 field_lengths)
        train_df_tip[col] = codes_tip.iloc[:len(train_df_tip)].values
        test_df_tip[col]  = codes_tip.iloc[len(train_df_tip):].values
# --- [新增结束] ---

# ==============================
# 7️⃣ (原 Step 6) 划分 train / val
# ==============================
print(f"Step 7/11: 划分 Train/Val (80/20)，基于 {new_label_col} 分层...")

if len(train_df) == 0:
    print("🔴 错误：train_df 在过滤后为空！无法进行 train_test_split。")
    print("    请检查您的 train_cleaned.csv 文件是否包含有效的 'image_path_final' 和 'label_4c'。")
    exit(1)

train_split, val_split = train_test_split(
    train_df, test_size=0.2, random_state=42,
    stratify=train_df[new_label_col] 
)
test_split = test_df # test_df 保持为 test_cleaned.csv 的全部内容
print(f"    Train: {len(train_split)}, Val: {len(val_split)}, Test: {len(test_split)}")
train_indices = train_split.index
val_indices = val_split.index

# ==============================
# 8️⃣ (原 Step 7) 标准化连续特征
# ==============================
print("Step 8/11: 标准化连续特征...")
scaler = StandardScaler()
if final_continuous_cols: 
    scaler.fit(train_split[final_continuous_cols])
    # (使用 .loc 避免 FutureWarning)
    train_split.loc[:, final_continuous_cols] = scaler.transform(train_split[final_continuous_cols])
    val_split.loc[:, final_continuous_cols]   = scaler.transform(val_split[final_continuous_cols])
    test_split.loc[:, final_continuous_cols]  = scaler.transform(test_split[final_continuous_cols])
else:
    print("    (跳过：未找到连续特征)")

# ==============================
# 9️⃣ (原 Step 8) 保存处理后的特征 & 字段长度
# ==============================
print("Step 9/11: 保存处理后的特征 CSV (无表头) 和 field_lengths.pt...")
# DVM-Car 脚本顺序: 连续型, 然后 分类型
ordered_feature_cols = final_continuous_cols + final_categorical_cols
# 为连续特征添加 '1' 到 field_lengths 的开头
# (field_lengths 仅包含分类特征的基数)
final_field_lengths = [1] * len(final_continuous_cols) + field_lengths

for df, name in zip([train_split, val_split, test_split], ["train", "val", "test"]):
    # 仅当 ordered_feature_cols 不为空时才保存
    if ordered_feature_cols:
        features_only_df = df[ordered_feature_cols]
        # 对应您列表中的 dvm_features_..._physical_jittered_50.csv
        out_path = os.path.join(OUT_DIR, f"{name}_features.csv")
        features_only_df.to_csv(out_path, index=False, header=False) 
        print(f"💾 Saved {name}_features.csv")
    else:
        print(f"    (跳过 {name}_features.csv：未找到任何特征)")

# 对应您列表中的 tabular_lengths_all_views_physical.pt
torch.save(final_field_lengths, os.path.join(OUT_DIR, "tabular_lengths.pt"))
print(f"💾 Saved tabular_lengths.pt (Features: {len(final_field_lengths)}, Lengths: {final_field_lengths})")

# ==============================
# 9️⃣.B [新增] 处理并保存 _TIP 版本
# ==============================
print("\n--- 正在处理并保存 _TIP (无填补) 版本 ---")

# 1. [TIP 划分] 使用 "填补版" 的索引来划分 "TIP版" 数据
print("    Step 7_TIP: 划分 _TIP 数据 (使用相同索引)...")
train_split_tip = train_df_tip.loc[train_indices]
val_split_tip = train_df_tip.loc[val_indices]
test_split_tip = test_df_tip # Test 集保持完整

# 2. [TIP 标准化] 重用之前 fit 好的 scaler 来 transform
#    (StandardScaler 会自动传播 NaNs，这正是我们想要的)
print("    Step 8_TIP: 标准化 _TIP 数据 (重用 scaler)...")
if final_continuous_cols: 
    train_split_tip.loc[:, final_continuous_cols] = scaler.transform(train_split_tip[final_continuous_cols])
    val_split_tip.loc[:, final_continuous_cols]   = scaler.transform(val_split_tip[final_continuous_cols])
    test_split_tip.loc[:, final_continuous_cols]  = scaler.transform(test_split_tip[final_continuous_cols])

# 3. [TIP 保存] 保存带 _TIP 后缀的 CSV
print("    Step 9_TIP: 保存 _TIP 特征 CSV...")
for df, name in zip([train_split_tip, val_split_tip, test_split_tip], ["train", "val", "test"]):
    if ordered_feature_cols:
        features_only_df_tip = df[ordered_feature_cols]
        # *** [修改] 添加 _TIP 后缀 ***
        out_path_tip = os.path.join(OUT_DIR, f"{name}_features_TIP.csv")
        features_only_df_tip.to_csv(out_path_tip, index=False, header=False) 
        print(f"💾 Saved {out_path_tip}")
    else:
        print(f"    (跳过 {name}_features_TIP.csv：未找到任何特征)")
print("--- _TIP 版本处理完毕 ---\n")


# ==============================
# 🔟 (原 Step 9) 保存标签 .pt 文件
# ==============================
print("Step 10/11: 保存 4-class 标签 .pt...")
def save_labels_4c(df, name):
    labels = df[new_label_col].astype(int).values
    # 对应您列表中的 labels_model_all_{name}_all_views.pt
    out_path = os.path.join(OUT_DIR, f"{name}_labels.pt")
    torch.save(torch.tensor(labels, dtype=torch.long), out_path)
    print(f"💾 Saved {out_path} (Classes: {np.unique(labels)})")

save_labels_4c(train_split, "train")
save_labels_4c(val_split, "val")
save_labels_4c(test_split, "test")

# ==============================
# 1️⃣1️⃣ (原 Step 10) 转换图像为 .NPY 并保存 .NPY 路径
# ==============================
print("Step 11/11: 转换图像为 .npy 并保存路径 .pt...")

def convert_and_save_paths(df, name):
    # 假设 'image_path_final' 是绝对 JPG 路径
    original_jpg_paths = df[image_path_col_final].tolist()
    npy_paths = [] 
    
    print(f"\n正在转换 {name} 图像 (.jpg -> .npy)...")
    for img_path in tqdm(original_jpg_paths, desc=f"Processing {name}"):
        if img_path is None or pd.isna(img_path):
            continue
            
        if not os.path.exists(img_path):
            print(f"⚠️ 路径不存在，跳过: {img_path}")
            continue
            
        # .npy 文件将被保存在 .jpg 旁边
        save_path = os.path.splitext(img_path)[0] + ".npy"
        
        try:
            if not os.path.exists(save_path):
                img = Image.open(img_path)
                img_np = np.array(img)
                # 确保目录存在 (以防万一)
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                np.save(save_path, img_np)
            
            npy_paths.append(save_path)
            
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
    
    # 对应您列表中的 {name}_paths_all_views.pt
    out_path = os.path.join(OUT_DIR, f"{name}_paths.pt")
    torch.save(npy_paths, out_path)
    print(f"💾 Saved {out_path} (包含 {len(npy_paths)} 条 .npy 路径)")

convert_and_save_paths(train_split, "train")
convert_and_save_paths(val_split, "val")
convert_and_save_paths(test_split, "test")

print(f"\n🎉 全部完成! 处理后的文件已准备就绪: {OUT_DIR}")

/tmp/ipykernel_188210/3885103284.py:222: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.30490534  0.30490534 -1.17317184 ...  0.30490534 -2.16027213
  0.30490534]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_split.loc[:, final_continuous_cols] = scaler.transform(train_split[final_continuous_cols])
/tmp/ipykernel_188210/3885103284.py:223: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.30490534  0.30490534  0.30490534  0.30490534  0.30490534  0.30490534
  0.30490534  0.30490534  0.30490534  0.30490534  0.30490534  0.30490534
 -2.54214304  0.30490534 -3.51997962  0.30490534  0.30490534  0.30490534
  0.30490534  0.30490534  0.30490534  0.30490534  0.30490534 -2.37539618
  0.30490534  0.30490534 -3.46336803  0.30490534  0.30490534  0.30490534
  0.30490534  0.30490534  0.30490534  0.30490534  0

数据源 (CSV): /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/
输出目录 (Processed): /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/features

Step 1/11: 加载已清理的 train_cleaned.csv 和 test_cleaned.csv...
    Train DF (loaded): 2864 行, Test DF (loaded): 422 行
Step 2/11: 定义特征类型 (基于您的要求)...
    (新) 使用您指定的 7 个连续特征: ['BitsAllocated', 'BitsStored', 'HighBit', 'LargestImagePixelValue', 'PixelRepresentation', 'SamplesPerPixel', 'SmallestImagePixelValue']
    (新) 使用您指定的 13 个分类特征: ['left_or_right_breast', 'image_view', 'BodyPartExamined', 'ConversionType', 'Modality', 'PhotometricInterpretation', 'SecondaryCaptureDeviceManufacturer', 'SecondaryCaptureDeviceManufacturerModelName', 'SeriesDescription', 'SpecificCharacterSet', 'breast_density', 'assessment', 'subtlety']
Step 3/11: 检查并删除 '恒定' 特征...
    -> 标记为删除 (Train 唯一值: 1, Test 唯一值: 1): BitsAllocated
    -> 标记为删除 (Train 唯一值: 1, Test 唯一值: 1): BitsStored
    -> 标记为删除 (Train 唯一值: 1, Test 唯一值: 1): HighBit
    -> 标记为删除 (Train 唯一值: 1, Test 唯一值: 1): Pix

Processing train: 100%|██████████| 2291/2291 [00:00<00:00, 117113.35it/s]


💾 Saved /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/features/train_paths.pt (包含 2291 条 .npy 路径)

正在转换 val 图像 (.jpg -> .npy)...


Processing val: 100%|██████████| 573/573 [00:00<00:00, 112658.14it/s]


💾 Saved /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/features/val_paths.pt (包含 573 条 .npy 路径)

正在转换 test 图像 (.jpg -> .npy)...


Processing test: 100%|██████████| 422/422 [00:00<00:00, 109414.37it/s]

💾 Saved /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/features/test_paths.pt (包含 422 条 .npy 路径)

🎉 全部完成! 处理后的文件已准备就绪: /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/features


In [1]:
import os
import pandas as pd
from PIL import Image

# --- 1. 配置 (从您的预处理脚本中复制) ---

# !! 确保这些路径与您的环境一致 !!
DATASET_ROOT = "/mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/" 
BASE_IMAGE_PATH = "/mnt/hdd/jiazy/"
TRAIN_CLEANED_CSV = os.path.join(DATASET_ROOT, "train_cleaned.csv")
image_path_col_final = "image_path_final" 

# --- 2. 辅助函数 (从您的预处理脚本中复制) ---

def make_abs_path(path):
    """构建绝对路径"""
    if pd.isna(path):
        return path
    # 使用 os.path.join 智能地拼接基础路径和相对路径
    return os.path.join(BASE_IMAGE_PATH, str(path))

# --- 3. 主检查逻辑 ---

def check_first_image_mode():
    print(f"--- 正在检查图像模式 ---")
    
    # 1. 加载 CSV
    try:
        df = pd.read_csv(TRAIN_CLEANED_CSV)
        print(f"成功加载: {TRAIN_CLEANED_CSV}")
    except FileNotFoundError:
        print(f"❌ 错误：找不到 CSV 文件。请检查路径：")
        print(f"{TRAIN_CLEANED_CSV}")
        return
    except Exception as e:
        print(f"❌ 错误：加载 CSV 时出错: {e}")
        return

    # 2. 构建绝对路径
    df[image_path_col_final] = df[image_path_col_final].apply(make_abs_path)
    
    # 3. 过滤掉无效路径
    df = df.dropna(subset=[image_path_col_final])
    if len(df) == 0:
        print("❌ 错误：在 CSV 中找不到任何有效的图像路径。")
        return

    # 4. 获取第一张图片的路径
    first_image_path = df[image_path_col_final].iloc[0]
    
    if not os.path.exists(first_image_path):
        print(f"❌ 错误：找到路径但文件不存在: {first_image_path}")
        print("请确保 BASE_IMAGE_PATH 设置正确。")
        return

    # 5. 打开图像并检查模式
    try:
        with Image.open(first_image_path) as img:
            mode = img.mode
            size = img.size
        
        print("\n" + "="*30)
        print(" 📊 图像诊断报告 (第一张图)")
        print("="*30)
        print(f" 文件路径: {first_image_path}")
        print(f" 图像尺寸: {size}")
        print(f" 图像模式: {mode}")
        print("="*30)

        # 6. 解释结果
        if mode == 'L':
            print("✅ 结论：这是 'L' 模式，即**灰度图 (Grayscale)**。")
            print("   (您应该在 transform 中使用 A.ToRGB() 将其转为 3 通道)")
        elif mode == 'RGB':
            print("✅ 结论：这是 'RGB' 模式，即**彩色图 (3 通道)**。")
            print("   (您不需要 A.ToRGB()，但使用它也是安全的)")
        elif mode == 'RGBA':
            print("✅ 结论：这是 'RGBA' 模式 (带透明度)。")
            print("   (A.ToRGB() 会自动处理它)")
        else:
            print(f"⚠️ 结论：这是一个不常见的模式 ('{mode}')。")
            print("   (强烈建议使用 A.ToRGB() 来标准化)")

    except Exception as e:
        print(f"❌ 错误：打开或检查图像时出错: {e}")


if __name__ == "__main__":
    check_first_image_mode()

--- 正在检查图像模式 ---
成功加载: /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/train_cleaned.csv

 📊 图像诊断报告 (第一张图)
 文件路径: /mnt/hdd/jiazy/cbis-ddsm-breast-cancer-image-dataset/jpeg/1.3.6.1.4.1.9590.100.1.2.148086015311392424203944628703986309919/1-064.jpg
 图像尺寸: (3800, 5640)
 图像模式: L
✅ 结论：这是 'L' 模式，即**灰度图 (Grayscale)**。
   (您应该在 transform 中使用 A.ToRGB() 将其转为 3 通道)
